In [ ]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=41009c17d2d249bea525bc739e56a04255a82d129a75ff86720b527d948fcd4a
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
import pandas as pd
import io
from google.colab import files
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [ ]:
uploaded = files.upload()

colunas = ['id', 'pSist', 'pDiast', 'qPA', 'pulso', 'fResp', 'grav', 'class']
df_treino = pd.read_csv(io.BytesIO(uploaded['env_vital_signals_4000.txt']), sep=',', names=colunas)
df_teste = pd.read_csv(io.BytesIO(uploaded['env_vital_signals_800.txt']), sep=',', names=colunas)

df_treino.head()
df_teste.head()

Saving env_vital_signals_800.txt to env_vital_signals_800.txt
Saving env_vital_signals_4000.txt to env_vital_signals_4000.txt


,id,pSist,pDiast,qPA,pulso,fResp,grav,class
0,0,15.701721,14.145122,8.721732,160.904175,20.926336,13.404672,1
1,1,18.907009,12.220855,8.733333,134.454047,17.972046,34.192304,2
2,2,14.698109,13.586879,8.614814,86.302237,4.168373,40.000000,2
3,3,16.428095,1.904802,-4.333333,138.880782,2.721256,27.122605,2
4,4,14.616501,13.700638,8.602531,51.356913,18.061923,47.188993,2


In [ ]:
df_treino.info()
df_teste.info()

df_treino.describe()
df_teste.describe()

df_treino.isnull().sum()
df_teste.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      4000 non-null   int64  
 1   pSist   4000 non-null   float64
 2   pDiast  4000 non-null   float64
 3   qPA     4000 non-null   float64
 4   pulso   4000 non-null   float64
 5   fResp   4000 non-null   float64
 6   grav    4000 non-null   float64
 7   class   4000 non-null   int64  
dtypes: float64(6), int64(2)
memory usage: 250.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      800 non-null    int64  
 1   pSist   800 non-null    float64
 2   pDiast  800 non-null    float64
 3   qPA     800 non-null    float64
 4   pulso   800 non-null    float64
 5   fResp   800 non-null    float64
 6   grav    800 non-null    float64
 7   class   800 non-null    int64 

id        0
pSist     0
pDiast    0
qPA       0
pulso     0
fResp     0
grav      0
class     0
dtype: int64

In [ ]:
# universos das variáveis de entrada
qPA_universe = np.linspace(-10, 10, 100)
pulso_universe = np.linspace(0, 200, 100)
fResp_universe = np.linspace(0, 22, 100)

# universo para variável de saída
grav_universe = np.linspace(0, 100, 100)

# variaveis de entrada
qPA = ctrl.Antecedent(qPA_universe, 'qPA')
pulso = ctrl.Antecedent(pulso_universe, 'pulso')
fResp = ctrl.Antecedent(fResp_universe, 'fResp')

# variável de saída
grav = ctrl.Consequent(grav_universe, 'grav')

# conjuntos fuzzy para qPA
qPA['baixa'] = fuzz.trimf(qPA_universe, [-10, -10, 0])
qPA['normal'] = fuzz.trimf(qPA_universe, [-10, 0, 10])
qPA['alta'] = fuzz.trimf(qPA_universe, [0, 10, 10])

# conjuntos fuzzy para pulso
pulso['baixo'] = fuzz.trimf(pulso_universe, [0, 0, 100])
pulso['normal'] = fuzz.trimf(pulso_universe, [0, 100, 200])
pulso['alto'] = fuzz.trimf(pulso_universe, [100, 200, 200])

# conjuntos fuzzy para fResp
fResp['baixa'] = fuzz.trimf(fResp_universe, [0, 0, 11])
fResp['normal'] = fuzz.trimf(fResp_universe, [0, 11, 22])
fResp['alta'] = fuzz.trimf(fResp_universe, [11, 22, 22])

# conjuntos fuzzy para grav
grav['critico'] = fuzz.trimf(grav_universe, [0, 0, 25])
grav['instavel'] = fuzz.trimf(grav_universe, [0, 25, 50])
grav['potencialmente_estavel'] = fuzz.trimf(grav_universe, [25, 50, 75])
grav['estavel'] = fuzz.trimf(grav_universe, [50, 75, 100])

'''# visualizar
qPA.view()
pulso.view()
fResp.view()
grav.view()'''

# definindo regras (PRIMEIRO FORAM SÓ 4 REGRAS. RMSE FICAVA MUITO ALTO)
regra1 = ctrl.Rule(qPA['baixa'] & pulso['baixo'] & fResp['baixa'], grav['critico'])
regra2 = ctrl.Rule(qPA['normal'] & pulso['normal'] & fResp['normal'], grav['estavel'])
regra3 = ctrl.Rule(qPA['baixa'] & pulso['alto'] & fResp['alta'], grav['instavel'])
regra4 = ctrl.Rule(qPA['baixa'] & pulso['alto'] & fResp['baixa'], grav['instavel'])
regra5 = ctrl.Rule(qPA['normal'] & pulso['alto'] & fResp['alta'], grav['instavel'])
regra6 = ctrl.Rule(qPA['alta'] & pulso['baixo'] & fResp['alta'], grav['instavel'])
regra7 = ctrl.Rule(qPA['alta'] | pulso['normal'] | fResp['normal'], grav['potencialmente_estavel'])
regra8 = ctrl.Rule(qPA['alta'] & pulso['normal'] & fResp['alta'], grav['potencialmente_estavel'])
regra9 = ctrl.Rule(qPA['normal'] & pulso['alto'] & fResp['alta'], grav['potencialmente_estavel'])
regra10 = ctrl.Rule(qPA['normal'] & pulso['normal'] & fResp['alta'], grav['potencialmente_estavel'])

# sistema de controle fuzzy
sistema_controle = ctrl.ControlSystem([regra1, regra2, regra3, regra4, regra5, regra6, regra7, regra8, regra9, regra10])
classificador_fuzzy = ctrl.ControlSystemSimulation(sistema_controle)

# TESTE aplicando
classificador_fuzzy.input['qPA'] = 2.5
classificador_fuzzy.input['pulso'] = 150
classificador_fuzzy.input['fResp'] = 15

# computando saída
classificador_fuzzy.compute()

# exibindo
print(classificador_fuzzy.output['grav'])

52.10161503967851


In [ ]:
from sklearn.model_selection import train_test_split

X = df_treino[['qPA', 'pulso', 'fResp']].values
y = df_treino['grav'].values

In [ ]:
# TREINO
# Inicialize o contador de erros
erro = 0
soma_diferenca_quadratica = 0
tolerancia = 1

# Loop sobre cada amostra
for i in range(len(X)):
    classificador_fuzzy.input['qPA'] = X[i][0]
    classificador_fuzzy.input['pulso'] = X[i][1]
    classificador_fuzzy.input['fResp'] = X[i][2]

    # Execute o sistema de controle fuzzy
    classificador_fuzzy.compute()

    # Obtenha a gravidade prevista
    gravidade_prevista = classificador_fuzzy.output['grav']

    # Obtenha a gravidade real
    gravidade_real = y[i]

    diferenca = gravidade_real - gravidade_prevista
    soma_diferenca_quadratica += diferenca ** 2

    # Verifique se a classe inferida é diferente da classe conhecida
    if abs(gravidade_prevista - gravidade_real) > tolerancia:
      erro += 1

# Calcule o número total de vítimas corretamente classificadas
C = len(X) - erro

rmse_total = np.sqrt(soma_diferenca_quadratica / len(X))

# Calcule o erro percentual total
erro_percentual_total = 100 * (C / len(X))

print(C)
print(len(X))
print(erro)

# Exibindo as métricas para todas as amostras
print(f'Total: RMSE = {rmse_total}, Erro Percentual Médio = {erro_percentual_total}%')

144
4000
3856
Total: RMSE = 19.644489583739432, Erro Percentual Médio = 3.5999999999999996%


In [ ]:
# VALIDAÇÃO CRUZADA
from sklearn.model_selection import KFold
from skfuzzy.control.controlsystem import ControlSystemSimulation

# Defina as configurações iniciais
n_folds = 5
kf = KFold(n_splits=n_folds)

# Lista para armazenar os RMSE de cada fold
rmse_scores = []

# Loop sobre cada fold
for train_index, val_index in kf.split(X):
    # Separe os dados em conjuntos de treinamento e validação
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Inicialize o contador de erros e a soma da diferença quadrática
    erro = 0
    soma_diferenca_quadratica = 0
    tolerancia = 1

    # Loop sobre cada amostra no conjunto de treinamento
    for i in range(len(X_train)):
        classificador_fuzzy.input['qPA'] = X_train[i][0]
        classificador_fuzzy.input['pulso'] = X_train[i][1]
        classificador_fuzzy.input['fResp'] = X_train[i][2]
        classificador_fuzzy.compute()

    # Loop sobre cada amostra no conjunto de validação
    for i in range(len(X_val)):
        classificador_fuzzy.input['qPA'] = X_val[i][0]
        classificador_fuzzy.input['pulso'] = X_val[i][1]
        classificador_fuzzy.input['fResp'] = X_val[i][2]
        classificador_fuzzy.compute()

        # Obtenha a gravidade prevista e a real
        gravidade_prevista = classificador_fuzzy.output['grav']
        gravidade_real = y_val[i]

        # Calcule a diferença quadrática
        diferenca = gravidade_real - gravidade_prevista
        soma_diferenca_quadratica += diferenca ** 2

        # Verifique se a classe inferida é diferente da classe conhecida
        if abs(gravidade_prevista - gravidade_real) > tolerancia:
            erro += 1

    # Calcule o RMSE e adicione à lista de scores
    rmse = np.sqrt(soma_diferenca_quadratica / len(X_val))
    rmse_scores.append(rmse)

# Exiba os resultados da validação cruzada
print("Scores de RMSE para cada fold:", rmse_scores)
print("RMSE médio:", np.mean(rmse_scores))

Scores de RMSE para cada fold: [19.841991531473642, 18.75776048015558, 19.340146773228184, 20.516710154106654, 19.722955562101774]
RMSE médio: 19.635912900213164


In [ ]:
# TESTE
X_teste = df_teste[['qPA', 'pulso', 'fResp']].values
y_teste = df_teste['grav'].values

In [ ]:
erro = 0
soma_diferenca_quadratica = 0
tolerancia = 1

# Loop sobre cada amostra
for i in range(len(X_teste)):
    classificador_fuzzy.input['qPA'] = X_teste[i][0]
    classificador_fuzzy.input['pulso'] = X_teste[i][1]
    classificador_fuzzy.input['fResp'] = X_teste[i][2]

    # Execute o sistema de controle fuzzy
    classificador_fuzzy.compute()

    # Obtenha a gravidade prevista
    gravidade_prevista = classificador_fuzzy.output['grav']

    # Obtenha a gravidade real
    gravidade_real = y_teste[i]

    diferenca = gravidade_real - gravidade_prevista
    soma_diferenca_quadratica += diferenca ** 2

    # Verifique se a classe inferida é diferente da classe conhecida
    if abs(gravidade_prevista - gravidade_real) > tolerancia:
      erro += 1

# Calcule o número total de vítimas corretamente classificadas
C = len(X_teste) - erro

rmse_total = np.sqrt(soma_diferenca_quadratica / len(X_teste))

# Calcule o erro percentual total
erro_percentual_total = 100 * (C / len(X_teste))

print(C)
print(len(X_teste))
print(erro)

# Exibindo as métricas para todas as amostras
print(f'Total: RMSE = {rmse_total}, Erro Percentual Médio = {erro_percentual_total}%')

28
800
772
Total: RMSE = 19.597896570735912, Erro Percentual Médio = 3.5000000000000004%
